This notebook gives a simple combination of literal matching and Named Entity Recognition using BERT (base model from huggingface).

The training phase of the BERT model was done in another kernel: Pytorch BERT for Named Entity Recognition.

In [ ]:
MAX_SAMPLE = None # set a small number for experimentation, set None for production.

# Install packages

In [ ]:
!pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
!pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
!pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

# Import

In [ ]:
import os
import re
import json
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

random.seed(123)
np.random.seed(456)

# Load data

In [ ]:
train_path = '../input/coleridgeinitiative-show-us-the-data/train.csv'
train = pd.read_csv(train_path)
train = train[:MAX_SAMPLE]
test_sample = train.sample(100)

paper_train_folder = '../input/coleridgeinitiative-show-us-the-data/train'
papers_train = {}
for paper_id in train['Id'].unique():
    with open(f'{paper_train_folder}/{paper_id}.json', 'r') as f:
        paper = json.load(f)
        papers_train[paper_id] = paper

In [ ]:
sample_submission_path = '../input/coleridgeinitiative-show-us-the-data/sample_submission.csv'
sample_submission = pd.read_csv(sample_submission_path)

papers_test = {}

paper_test_folder = '../input/coleridgeinitiative-show-us-the-data/test'
for paper_id in sample_submission['Id']:
    with open(f'{paper_test_folder}/{paper_id}.json', 'r') as f:
        paper = json.load(f)
        papers_test[paper_id] = paper

# Making validation set

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def totally_clean_text(txt):
    txt = clean_text(txt)
    txt = re.sub(' +', ' ', txt)
    return txt

In [ ]:
all_labels = {}

for label_1, label_2, label_3 in train[['dataset_title', 'dataset_label', 'cleaned_label']].itertuples(index=False):
    all_labels[label_1.lower()] = label_3.lower()
    all_labels[label_2.lower()] = label_3.lower()
    all_labels[label_3.lower()] = label_3.lower()

print(f'No. different labels: {len(all_labels)}')


validation_labels = ['adni', "alzheimer's disease neuroimaging initiative (adni)",
                     'alzheimers disease neuroimaging initiative', 'alzheimer s disease neuroimaging initiative adni ',
#                     'rural-urban continuum codes', 'rural urban continuum codes',
#                     'baccalaureate and beyond', 'baccalaureate and beyond longitudinal study',
                    ]
for val_lab in validation_labels:
    all_labels.pop(val_lab)
    
print(print(f'No. different labels: {len(all_labels)}'))

In [ ]:
papers_valid = {}
count_valid = 0
for i, id, dataset_label, cleaned_label in train[['Id', 'dataset_label', 'cleaned_label']].itertuples(): #test_sample['Id']: sample_submission['Id']
    paper = papers_train[id]
    
    labels = set()
    
    text_1 = '. '.join(section['text'] for section in paper).lower()
    text_2 = totally_clean_text(text_1)
    
    if dataset_label.lower() in validation_labels or cleaned_label.lower() in validation_labels:
        papers_valid[id] = papers_train[id]
        count_valid += 1
    else:
        for label in validation_labels:
            if label in text_1 or label in text_2:
                papers_valid[id] = papers_train[id]
                count_valid += 1
                break
    
#     if dataset_label.lower() in validation_labels or cleaned_label.lower() in validation_labels:
#         papers_valid[id] = papers_train[id]
#         count_valid += 1
            
for paper_id in papers_valid:
    papers_train.pop(paper_id)

print("Valid label count: ", count_valid)
print(len(papers_train))
print(len(papers_valid))



# Literal matching

### Create a knowledge bank

In [ ]:
# all_labels = {}

# for label_1, label_2, label_3 in train[['dataset_title', 'dataset_label', 'cleaned_label']].itertuples(index=False):
#     all_labels[label_1.lower()] = label_3.lower()
#     all_labels[label_2.lower()] = label_3.lower()
#     all_labels[label_3.lower()] = label_3.lower()
# #     all_labels.add(str(label_1).lower())
# #     all_labels.add(str(label_2).lower())
# #     all_labels.add(str(label_3).lower())
# # for la in zip(all_labels.keys(), all_labels.values()):
# #     print(la)
# print(f'No. different labels: {len(all_labels)}')

### Matching on test data

In [ ]:
# def clean_text(txt):
#     return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

# def totally_clean_text(txt):
#     txt = clean_text(txt)
#     txt = re.sub(' +', ' ', txt)
#     return txt

In [ ]:
# literal_preds = []

# for paper_id in papers_valid.keys(): #test_sample['Id']: sample_submission['Id']
#     paper = papers_valid[paper_id]
#     text_1 = '. '.join(section['text'] for section in paper).lower()
#     text_2 = totally_clean_text(text_1)
    
#     labels = set()
#     for label in all_labels.keys():
#         if label in text_1 or label in text_2:
#             labels.add(all_labels[label])
# #             if label != all_labels[label]:
# #                 print(f"{label} -> {all_labels[label]}")
# #     print(labels)
    
#     labels = sorted(labels, key=len, reverse = True)
# #     print(labels)
# #     for idx, lab1 in enumerate(labels):
# # #         print(lab1)
# # #         print("...")
# #         for lab2 in [label for label in labels[idx:] if label != lab1]:
# # #             print(lab2)
# #             words1 = set(lab1.split())
# #             words2 = set(lab2.split())
# # #             print(f"{len(words1 & words2)}/{len(words1)}: {len(words1 & words2)/len(words1)}")
# #             if len(words1 & words2)/len(words1) > 0.75:
# #                 labels.remove(lab2)
        
# #         print("---")
#     literal_preds.append('|'.join(labels))
# #     print(literal_preds[-1])
# #     print("***")
    
# # print(literal_preds)


In [ ]:
# random.sample(list(zip(test_sample['cleaned_label'], literal_preds)), 8)

# Bert prediction

### Paths and Hyperparameters

In [ ]:
MAX_LENGTH = 48 # max no. words for each sentence.
OVERLAP = 16 # if a sentence exceeds MAX_LENGTH, we split it to multiple sentences with overlapping

PREDICT_BATCH = 64000 

PRETRAINED_PATH = '../input/scibert-validation-set-version/output'
TEST_INPUT_SAVE_PATH = './input_data'
TEST_NER_DATA_FILE = 'test_ner_input.json'
TRAIN_PATH = '../input/coleridge-bert-models/train_ner.json'
VAL_PATH = '../input/coleridge-bert-models/train_ner.json'

PREDICTION_SAVE_PATH = './pred'
PREDICTION_FILE = 'test_predictions.txt'

### Transform data to NER format

Group by publication, training labels should have the same form as expected output.

In [ ]:
train = train.groupby('Id').agg({
    'pub_title': 'first',
    'dataset_title': '|'.join,
    'dataset_label': '|'.join,
    'cleaned_label': '|'.join
}).reset_index()

print(f'No. grouped training rows: {len(train)}')

In [ ]:
def clean_training_text(txt):
    """
    similar to the default clean_text function but without lowercasing.
    """
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt)).strip()

def shorten_sentences(sentences):
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

In [ ]:
# test_rows = [] # test data in NER format
# paper_length = [] # store the number of sentences each paper has

# for paper_id in papers_valid.keys():
#     # load paper
#     paper = papers_valid[paper_id]

#     # extract sentences
#     sentences = [clean_training_text(sentence) for section in paper 
#                  for sentence in section['text'].split('.')
#                 ]
#     sentences = shorten_sentences(sentences) # make sentences short
#     sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
#     sentences = [sentence for sentence in sentences if any(word in sentence.lower() for word in ['data', 'study'])]

#     # collect all sentences in json
#     for sentence in sentences:
#         sentence_words = sentence.split()
#         dummy_tags = ['O']*len(sentence_words)
#         test_rows.append({'tokens' : sentence_words, 'tags' : dummy_tags})

#     # track which sentence belongs to which data point
#     paper_length.append(len(sentences))

# print(f'total number of sentences: {len(test_rows)}')

###################################################################################

#Import preselection model
from joblib import dump, load
clf = load('../input/preselection-model-validation-set/preselection_model.joblib') 

test_rows = [] # test data in NER format
paper_length = [] # store the number of sentences each paper has



for paper_id in papers_valid.keys():
    # load paper
    paper = papers_valid[paper_id]
    
    # extract sentences
    sentences = [clean_training_text(sentence) for section in paper 
                 for sentence in section['text'].split('.')
                ]
    sentences = shorten_sentences(sentences) # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    
    predictions = clf.predict(sentences)
#     print(sum(predictions))
    sentences = [sentence for idx, sentence in enumerate(sentences) if predictions[idx] > 0.0]
#     sentences = [sentence for sentence in sentences if re.match( r'(.* has been used.*|.* (dataset|survey|database|study) .*|.*obtained from .*|.*data from .*|.*data used in .*)', sentence.lower())]#any(word in sentence.lower() for word in ['national', 'research', 'data', 'survey', 'science', 'technology', 'development', 'department', 'study', 'statistics','international'])]
        
    # collect all sentences in json
    for sentence in sentences:
        sentence_words = sentence.split()
        dummy_tags = ['O']*len(sentence_words)
        test_rows.append({'tokens' : sentence_words, 'tags' : dummy_tags})
    
    # track which sentence belongs to which data point
    paper_length.append(len(sentences))
    
print(f'total number of sentences: {len(test_rows)}')

### Do predict and collect results

In [ ]:
os.environ["MODEL_PATH"] = f"{PRETRAINED_PATH}"
os.environ["TRAIN_FILE"] = f"{TRAIN_PATH}"
os.environ["VALIDATION_FILE"] = f"{VAL_PATH}"
os.environ["TEST_FILE"] = f"{TEST_INPUT_SAVE_PATH}/{TEST_NER_DATA_FILE}"
os.environ["OUTPUT_DIR"] = f"{PREDICTION_SAVE_PATH}"

In [ ]:
# copy my_seqeval.py to the working directory because the input directory is non-writable
!cp /kaggle/input/coleridge-packages/my_seqeval.py ./

# make necessart directories and files
os.makedirs(TEST_INPUT_SAVE_PATH, exist_ok=True)

In [ ]:
def bert_predict():
    !python ../input/kaggle-ner-utils/kaggle_run_ner.py \
    --model_name_or_path "$MODEL_PATH" \
    --train_file "$TRAIN_FILE" \
    --validation_file "$VALIDATION_FILE" \
    --test_file "$TEST_FILE" \
    --output_dir "$OUTPUT_DIR" \
    --report_to 'none' \
    --seed 123 \
    --do_predict

In [ ]:
bert_outputs = []

for batch_begin in range(0, len(test_rows), PREDICT_BATCH):
    # write data rows to input file
    with open(f'{TEST_INPUT_SAVE_PATH}/{TEST_NER_DATA_FILE}', 'w') as f:
        for row in test_rows[batch_begin:batch_begin+PREDICT_BATCH]:
            json.dump(row, f)
            f.write('\n')
    
    # remove output dir
    !rm -r "$OUTPUT_DIR"
    
    # do predict
    bert_predict()
    
    # read predictions
    with open(f'{PREDICTION_SAVE_PATH}/{PREDICTION_FILE}') as f:
        this_preds = f.read().split('\n')[:-1]
        bert_outputs += [pred.split() for pred in this_preds]

### Restore Dataset labels from predictions

In [ ]:
# get test sentences
test_sentences = [row['tokens'] for row in test_rows]

del test_rows

In [ ]:
bert_dataset_labels = [] # store all dataset labels for each publication

for length in paper_length:
    labels = set()
    for sentence, pred in zip(test_sentences[:length], bert_outputs[:length]):
        curr_phrase = ''
        for word, tag in zip(sentence, pred):
            if tag == 'B': # start a new phrase
                if curr_phrase:
                    labels.add(curr_phrase)
                    curr_phrase = ''
                curr_phrase = word
            elif tag == 'I' and curr_phrase: # continue the phrase
                curr_phrase += ' ' + word
            else: # end last phrase (if any)
                if curr_phrase:
                    labels.add(curr_phrase)
                    curr_phrase = ''
        # check if the label is the suffix of the sentence
        if curr_phrase:
            labels.add(curr_phrase)
            curr_phrase = ''
    
    # record dataset labels for this publication
    bert_dataset_labels.append(labels)
    
    del test_sentences[:length], bert_outputs[:length]

In [ ]:
bert_dataset_labels[:100]

### Filter based on Jaccard score and clean

In [ ]:
def jaccard_similarity(s1, s2):
    l1 = s1.split(" ")
    l2 = s2.split(" ")    
    intersection = len(list(set(l1).intersection(l2)))
    union = (len(l1) + len(l2)) - intersection
    return float(intersection) / union

filtered_bert_labels = []

for labels in bert_dataset_labels:
    filtered = []
    
    for label in sorted(labels, key=len):
        label = clean_text(label)
        if len(filtered) == 0 or all(jaccard_similarity(label, got_label) < 0.75 for got_label in filtered):
            filtered.append(label)
    filtered_bert_labels.append('|'.join(filtered))

In [ ]:
filtered_bert_labels[:20]
print([x for x in filtered_bert_labels if x != ''])

# Aggregate final predictions and write submission file

In [ ]:
final_predictions = []
for bert_pred in filtered_bert_labels:
    if bert_pred:
        final_predictions.append(bert_pred)
    else:
        final_predictions.append('')

In [ ]:
# print(final_predictions)
print([x for x in final_predictions if x != ''])

In [ ]:
# sample_submission['PredictionString'] = final_predictions
# sample_submission.head()

In [ ]:
# sample_submission.to_csv(f'submission.csv', index=False)